In [ ]:
#import packages
import pandas as pd         
import gc                         
gc.enable()  

In [ ]:
orders = pd.read_csv('../input/instacart-market-basket-analysis/orders.csv' )
order_products_train = pd.read_csv('../input/instacart-market-basket-analysis/order_products__train.csv')
order_products_prior = pd.read_csv('../input/instacart-market-basket-analysis/order_products__prior.csv')
products = pd.read_csv('../input/instacart-market-basket-analysis/products.csv')

In [ ]:
#orders = orders.loc[orders.user_id.isin(orders.user_id.drop_duplicates().sample(frac=0.1, random_state=25))] 

In [ ]:
orders.head()

In [ ]:
order_products_train.head()

In [ ]:
order_products_prior.head()

In [ ]:
products.head()

In [ ]:
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

In [ ]:
op = orders.merge(order_products_prior, on='order_id', how='inner')
op.head()

1.User variables

In [ ]:
#Number of orders per customer
user = op.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user = user.reset_index()
user.head()

In [ ]:
#How frequent a customer has reordered products
u_reorder = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio')
u_reorder = u_reorder.reset_index()
u_reorder.head()

In [ ]:
#Keep las 5 orders
op['order_number_back'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1 
op.head()

In [ ]:
#Keep last 5 orders
op5 = op[op.order_number_back <= 5]
op5.head()

In [ ]:
#Last 5 oders of each user and product_id
last_five = op5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five.head()

In [ ]:
#Size of orders for each user
u_last_five = last_five.groupby ('user_id')[["times_last5"]].count()
u_last_five.columns = ['size_last5']
u_last_five.head()

In [ ]:
#Mean of last 5 orders of each customer
#u_last_five ['mean_size_last5']= u_last_five.size_last5 / 5
#u_last_five.head()

In [ ]:
#Max days of 5 last orders for each user
#u_last_five ['max_days_5'] = op5.groupby ('user_id') [["days_since_prior_order"]].max()
#u_last_five.head()

In [ ]:
#Mean days of 5 last orders for each user
#u_last_five ['mean_days_5'] = op5.groupby ('user_id') [["days_since_prior_order"]].mean()
#u_last_five.head()

In [ ]:
#Merge user with u_reorder
user = user.merge(u_reorder, on='user_id', how='left')
del u_reorder
gc.collect()
user.head()

In [ ]:
#Merge user with last_five 
user = user.merge (u_last_five, on = "user_id" , how ="left")
del u_last_five
gc.collect()
user.head()

2.Product variables

In [ ]:
#Number of purchases for each product
prd = op.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
prd = prd.reset_index()
prd.head()

In [ ]:
#Remove products with less than 40 purchases
p_reorder = op.groupby('product_id').filter(lambda x: x.shape[0] >40)
p_reorder.head()

In [ ]:
#Mean of reorders
p_reorder = p_reorder.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorder = p_reorder.reset_index()
p_reorder.head()

In [ ]:
#Mean position of each product
p_reorder['avg_position'] = op.groupby ('product_id')[["add_to_cart_order"]].mean()
p_reorder.head()

In [ ]:
#Probability of reordered for each product within last 5 orders of the users
op5_ratio = op5.groupby ('product_id')["reordered"].mean().to_frame ('p_reorder_last5')
op5_ratio = op5_ratio.reset_index()
op5_ratio.head()

In [ ]:
#Merge the prd DataFrame with p_reorder
prd = prd.merge(p_reorder, on='product_id', how='left')
del p_reorder
gc.collect()
prd.head()

In [ ]:
#Merge the prd DataFrame with op5
prd = prd.merge(op5_ratio, on='product_id', how='left')
gc.collect()
prd.head()

In [ ]:
#Fill NaN Values
prd['p_reorder_ratio'] = prd['p_reorder_ratio'].fillna(value=0)
prd['avg_position'] = prd ['avg_position'].fillna(value=0)
prd['p_reorder_last5'] = prd['p_reorder_last5'].fillna(value=0)
prd.head()

3.User-product variables

In [ ]:
#How many times a user bought a product
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp = uxp.reset_index()
uxp.head()

In [ ]:
#Calculate Times_Bought_N
times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times.head()

In [ ]:
#Calculate Order_Range_D (1)
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()

In [ ]:
#Calculate Order_Range_D (2)
first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()


In [ ]:
#Find Order_Range_D
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
span.head()

In [ ]:
#Find the final ratio
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D
uxp_ratio.head()

In [ ]:
#Delete DataFrames
uxp_ratio = uxp_ratio.drop(['Times_Bought_N', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()

In [ ]:
#Delete DataFrames
del [times, first_order_no, span]
gc.collect()

In [ ]:
#Merge with uxp
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')
del uxp_ratio
uxp.head()

In [ ]:
#How many times a user bought a product on its last 5 orders (already exist)
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')
uxp.head()

In [ ]:
#Fill NaN values
uxp = uxp.fillna(0)
uxp.head()

In [ ]:
#Ratio of last 5 orders of each customer for a product
uxp ['last5_ ratio']= uxp.times_last5 / 5
uxp.head()

In [ ]:
#Max days of 5 last orders for each product of users
uxp['max_days_last5'] = op5.groupby('user_id')[["days_since_prior_order"]].max()
uxp.head()

In [ ]:
uxp = uxp.fillna(0)

Merge features

In [ ]:
#Merge uxp features with the user features
data = uxp.merge(user, on='user_id', how='left')
data.head()

In [ ]:
#Merge uxp & user features with prd features
data = data.merge(prd, on='product_id', how='left')
data.head()

In [ ]:
#Delete DataFrames
del op, user, prd, uxp, last_five, op5, total_orders
gc.collect()

Train and Test DataFrames

In [ ]:
#keep only the future orders from all customers: train & test 
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

In [ ]:
# bring the info of the future orders to data DF
data = data.merge(orders_future, on='user_id', how='left')
del orders_future
gc.collect()
data.head()

In [ ]:
#Keep only the customers who we know what they bought in their future order
data_train = data[data.eval_set=='train']
data_train.head()

In [ ]:
#Get from order_products_train all the products that the train users bought bought in their future order
data_train = data_train.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
del order_products_train
del order_products_prior
gc.collect()
data_train.head()

In [ ]:
#Fill NaN values
data_train['reordered'] = data_train['reordered'].fillna(0)
data_train.head()

In [ ]:
#We set user_id and product_id as the index of the DF
data_train = data_train.set_index(['user_id', 'product_id'])
data_train.head()

In [ ]:
#We remove all non-predictor variables
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train.head()

In [ ]:
#Keep only the future orders from customers who are labelled as test
data_test = data[data.eval_set=='test']
data_test.head()

In [ ]:
#We set user_id and product_id as the index of the DF
data_test = data_test.set_index(['user_id', 'product_id'])
data_test.head()

In [ ]:
#We remove all non-predictor variables
data_test = data_test.drop(['eval_set','order_id'], axis=1)
data_test.head()

In [ ]:
#Delete Dataframe
del data
gc. collect()

Create predictive model (fit)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data_train.drop('reordered', axis=1), data_train.reordered, test_size=0.8, random_state=42)

parameters = {'max_depth':6,
              'colsample_bytree':0.4,
              'subsample':0.75,
              'eval_metric':'logloss',
              }
xgbc = xgb. XGBClassifier (objective='binary:logistic', parameters = parameters, num_boost_round=10)

model = xgbc.fit (X_train, y_train)
xgb.plot_importance(model)

In [ ]:
model.get_params()

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
paramGrid = {"max_depth":[5,10],
            "colsample_bytree":[0.3,0.4]}  

xgbc = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', num_boost_round=10)

gridsearch = GridSearchCV(xgbc, paramGrid, cv=3, verbose=2, n_jobs=1)

model = gridsearch.fit(X_train, y_train)

print("The best parameters are: /n",  gridsearch.best_params_)

model = gridsearch.best_estimator_

# Delete X_train , y_train
del [X_train, y_train]


In [ ]:
#Retrieve the parametrs
model.get_xgb_params()

Apply predictive model (predict)

In [ ]:
## OR set a custom threshold (in this problem, 0.21 yields the best prediction)
test_pred = (model.predict_proba(data_test)[:,1] >= 0.21).astype(int)
test_pred[0:20] #display the first 20 predictions of the numpy array

In [ ]:
#Save the prediction (saved in a numpy array) on a new column in the data_test DF
data_test['prediction'] = test_pred
data_test.head()

In [ ]:
# Reset the index
final = data_test.reset_index()
# Keep only the required columns to create our submission file (for chapter 6)
final = final[['product_id', 'user_id', 'prediction']]
del data_test
gc.collect()
final.head()

Creation of Submission File

In [ ]:
#Keep test orders
orders_test = orders.loc[orders.eval_set=='test',("user_id", "order_id") ]
orders_test.head()

In [ ]:
#Merge with final
final = final.merge(orders_test, on='user_id', how='left')
final.head()

In [ ]:
#remove user_id column
final = final.drop('user_id', axis=1)

In [ ]:
#convert product_id as integer
final['product_id'] = final.product_id.astype(int)

## Remove all unnecessary objects
del orders
del orders_test
del products
gc.collect()

final.head()

In [ ]:
d = dict()
for row in final.itertuples():
    if row.prediction== 1:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in final.order_id:
    if order not in d:
        d[order] = 'None'
        
gc.collect()

#We now check how the dictionary were populated (open hidden output)
d

In [ ]:
#Convert the dictionary into a DataFrame
sub = pd.DataFrame.from_dict(d, orient='index')

#Reset index
sub.reset_index(inplace=True)
#Set column names
sub.columns = ['order_id', 'products']

sub.head()

In [ ]:
#Check if sub file has 75000 predictions
sub.shape[0]
print(sub.shape[0]==75000)

In [ ]:
sub.to_csv('sub.csv', index=False)